# Imports

In [ ]:
# Suprress warnings
import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")

    # Standard imports
    import matplotlib
    import matplotlib.pyplot as plt
    import matplotlib.cm as cm
    import numpy as np

    # Drake imports
    from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
    import pydrake
    from pydrake.all import LogOutput, DirectCollocation, DirectTranscription, MathematicalProgram, InputPortSelection

# Imports of other project files
from log_wrapper import LogWrapper
import constants
import finger
import pedestal
from paper import Paper

from pydrake.all import (MultibodyPlant, Parser, DiagramBuilder, Simulator,
                         PlanarSceneGraphVisualizer, SceneGraph, TrajectorySource,
                         SnoptSolver, MultibodyPositionToGeometryPose, PiecewisePolynomial,
                         MathematicalProgram, JacobianWrtVariable, eq, RollPitchYaw, AutoDiffXd, BodyIndex)

# Other imports
import importlib

import scipy.interpolate

In [ ]:
# Matplotlib configuring
plt.style.use(['science', 'no-latex'])
font = {'size'   : 14}
matplotlib.rc('font', **font)

In [ ]:
# Meshcat init
proc, zmq_url, web_url = start_zmq_server_as_subprocess()

# Simulation setup
## Pre-finalize steps

In [ ]:
builder = pydrake.systems.framework.DiagramBuilder()

# Add all elements
plant, scene_graph = pydrake.multibody.plant.AddMultibodyPlantSceneGraph(builder, time_step=constants.DT)
v_stiction=1e-3
plant.set_stiction_tolerance(v_stiction)
pedestal_instance = pedestal.AddPedestal(plant)

# These joint angles start the paper approximately the right spot, no matter how many links are used
num_links = 2
# def_joint_angles = list(np.zeros(num_links))
# def_joint_angles[0] = np.pi/100
# def_joint_angles[1] = np.pi/100
# def_joint_angles[int(num_links*0.65)-2] = -np.pi/10
# def_joint_angles[int(num_links*0.65)-1] = -np.pi/10
# def_joint_angles[int(num_links*0.65)] = -np.pi/10
# def_joint_angles[int(num_links*0.65)+1] = -np.pi/10
# # def_joint_angles[int(num_links*0.65)+2] = -np.pi/10
def_joint_angles = 0#np.pi/3

paper = Paper(plant, scene_graph, num_links, "NATURAL", default_joint_angle=def_joint_angles,
              stiffness=0,#2.5e-2,
              damping=0,)#7.12547340446979e-06)
paper.weld_paper_edge(pedestal.PEDESTAL_WIDTH, pedestal.PEDESTAL_HEIGHT)

finger_instance, finger_body = finger.AddFinger(plant, constants.INIT_Y, constants.INIT_Z)

# Set up logger (needs to happen after all bodies are added)
log_wrapper = LogWrapper(plant.num_bodies(), int(finger_body.index()), paper.get_free_edge_idx(), paper.joint_idxs)
builder.AddSystem(log_wrapper)

Calculate link inertia to use in damping calculations:

In [ ]:
paper.plant.get_body(BodyIndex(paper.get_free_edge_idx())).default_rotational_inertia().CalcPrincipalMomentsOfInertia()[0]

## Controller selection

In [ ]:
## CHOOSE CONTROL SYSTEM HERE BY UNCOMMENTING
# # PD control: hits too low
# finger_ctrlr = finger.PDFinger(
#     plant,
#     int(finger_instance),
#     [
#         [constants.INIT_Y, constants.INIT_Z],
#         [constants.INIT_Y*1.1, constants.INIT_Z],
#         [constants.INIT_Y*1.1, pedestal.PEDESTAL_HEIGHT+constants.FINGER_RADIUS/2+0.05],
#         [0, pedestal.PEDESTAL_HEIGHT+constants.FINGER_RADIUS/2+0.05],
#         [0, pedestal.PEDESTAL_HEIGHT+constants.FINGER_RADIUS/2],
#     ],
#     tspan_per_segment=1,
#     ky=10,
#     kz=10
# )

# PD control: hits too high
# finger_ctrlr = finger.PDFinger(
#     plant,
#     int(finger_instance),
#     [
#         [constants.INIT_Y, constants.INIT_Z],
#         [constants.INIT_Y*1.1, constants.INIT_Z],
#         [constants.INIT_Y*1.1, pedestal.PEDESTAL_HEIGHT+constants.FINGER_RADIUS/2+0.08],
#         [0, pedestal.PEDESTAL_HEIGHT+constants.FINGER_RADIUS/2+0.08],
#         [0, pedestal.PEDESTAL_HEIGHT+constants.FINGER_RADIUS/2],
#     ],
#     tspan_per_segment=1,
#     ky=10,
#     kz=10
# )


# Edge feedback
finger_ctrlr = finger.EdgeController(
    plant,
    paper,
    int(finger_instance),
    F_Nd=2,
    debug=True,
)

# # Optimization controller
# finger_ctrlr = finger.OptimizationController(
#     plant,
#     paper,
#     int(finger_instance),
#     paper.get_free_edge_instance()
# )

# Blank controller
# finger_ctrlr = finger.BlankController(
#     plant,
#     int(finger_instance)
# )

## Post-finalize steps

In [ ]:
plant.Finalize()

## Post finalize steps
# Conect finger controller
builder.AddSystem(finger_ctrlr)
builder.Connect(finger_ctrlr.get_output_port(), plant.get_actuation_input_port(finger_instance))
builder.Connect(plant.get_body_poses_output_port(), finger_ctrlr.get_input_port(0))
builder.Connect(plant.get_body_spatial_velocities_output_port(), finger_ctrlr.get_input_port(1))

# Add logger
builder.Connect(plant.get_body_poses_output_port(), log_wrapper.get_input_port(0))
builder.Connect(plant.get_body_spatial_velocities_output_port(), log_wrapper.get_input_port(1))
builder.Connect(plant.get_body_spatial_accelerations_output_port(), log_wrapper.get_input_port(2)) 
builder.Connect(plant.get_contact_results_output_port(), log_wrapper.get_input_port(3))
builder.Connect(plant.get_reaction_forces_output_port(), log_wrapper.get_input_port(4))

# Visualization and logging
logger = LogOutput(log_wrapper.get_output_port(), builder)
vis = pydrake.systems.meshcat_visualizer.ConnectMeshcatVisualizer(builder, scene_graph)
vis.set_planar_viewpoint(camera_position=[1, 0, 0], xmin=-0.3, xmax=0.3, ymin=-0.3, ymax=0.3)

# Build diagram and do actions requiring 
diagram = builder.Build()
diagram_context = diagram.CreateDefaultContext()

In [ ]:
if type(finger_ctrlr) is finger.OptimizationController:
    finger_ctrlr.optimize(plant.GetPositions(diagram_context))

# Run simulation

In [ ]:
# Finalize simulation and visualization
simulator = pydrake.systems.analysis.Simulator(diagram, diagram_context)
simulator.Initialize()
vis.start_recording()
try:
    simulator.AdvanceTo(constants.TSPAN)
except RuntimeError as e:
    print(e)

# This way, we can use the actual time the simulation ran for
effective_tspan = max(logger.sample_times())

vis.stop_recording()
vis.publish_recording()

# Plots used in orginal paper

In [ ]:
# Plot manipulator position vs. trajectory
# PROGRAMMING: Use body index for finger instead of model instance
if type(finger_ctrlr) is finger.PDFinger:
    plt.figure(figsize=(2*3,2*2))
    plt.plot(logger.data()[log_wrapper.entries_per_body*int(finger_instance)+1],
             logger.data()[log_wrapper.entries_per_body*int(finger_instance)+2],
             label='Manipulator position')
    plt.plot(finger_ctrlr.ys, finger_ctrlr.zs, label='Trajectory')
    plt.xlabel("$y$ position")
    plt.ylabel("$z$ position")
    plt.legend()
    plt.show()

In [ ]:
# Generate plots for paper
nb = plant.num_bodies()
y_traces = []
z_traces = []
theta_traces = []
for b in paper.link_idxs:
    y_traces.append(logger.data()[log_wrapper.entries_per_body*b+1])
    z_traces.append(logger.data()[log_wrapper.entries_per_body*b+2])
    theta_traces.append(logger.data()[log_wrapper.entries_per_body*b+3])
y_traces = np.array(y_traces)
z_traces = np.array(z_traces)
theta_traces = np.array(theta_traces)

times_ = np.arange(0,effective_tspan, effective_tspan/10)
cmap = cm.get_cmap("viridis_r")
plt.figure(figsize=(2*3,2*2))
if type(finger_ctrlr) is finger.PDFinger:
    plt.plot(finger_ctrlr.ys, finger_ctrlr.zs, '--k', zorder=-1)
for t in times_:
    c = cmap(t/effective_tspan)
    idx = np.argmax(logger.sample_times() >= t)
    
    # Plot paper
    # PROGRAMMING: Account for paper thickness in plots
    for y, z, theta in zip(y_traces[:,idx], z_traces[:,idx], theta_traces[:,idx]):
        y0 = y - np.cos(theta)*paper.link_width/2
        z0 = z - np.sin(theta)*paper.link_width/2
        y1 = y + np.cos(theta)*paper.link_width/2
        z1 = z + np.sin(theta)*paper.link_width/2
        plt.plot([y0, y1], [z0, z1], color=c)
        
    
    # Plot manipulator
    plt.scatter(logger.data()[log_wrapper.entries_per_body*int(finger_instance)+1,idx], 
                logger.data()[log_wrapper.entries_per_body*int(finger_instance)+2,idx],
                color=c, s=300, zorder=1)

xlim = plt.xlim()
ylim = plt.ylim()
plt.scatter([xlim[0]-50, xlim[0]-50], [ylim[0]-50, ylim[0]-50], c=[0, effective_tspan], cmap=cmap)
plt.xlim(xlim)
plt.ylim(ylim)
cb = plt.colorbar()
cb.set_label("Time")
plt.xlabel("$y$ position")
plt.ylabel("$z$ position")
plt.show()

This is a sanity check to see how much force we're neglecting by ignoring drag. The colored lines are the drag forces at each link, and the dashed black line is the force due to gravity for scale.

Note that the formula for drag force is $\rho v^2 C_d A_{paper}$.

$C_d$ comes from [this site](https://www.engineersedge.com/fluid_flow/rectangular_flat_plate_drag_14036.htm), which says it depends on $L/d$. For 20 links, $L/d\approx15$. Rounding this down to 10, we get $C_d=1.22$.

In [ ]:
# Plot drag forces
plt.figure(figsize=(16, 12))
nb = plant.num_bodies
vel_traces = []
for b in paper.link_idxs:
    rho = 1.225
    C_d = paper.link_width*paper.depth
    vel_sqred = logger.data()[log_wrapper.entries_per_body*b+7]**2+logger.data()[log_wrapper.entries_per_body*b+8]**2
    f = 0.5*rho*C_d*vel_sqred
    plt.plot(f)

plt.axhline(paper.link_mass*9.81, color='black', linestyle='--')
plt.show()

# Careful debug plots
What do we want to verify?

If the net forces and torques on each object match my expectations, then I will be satisfied.

We can use free body diagrams to determine the expected values, then compare to sim. First, we will do forces on each object, then torques on each object.

# Forces on manipulator

## Expected values
<img src=forces.png width=500px>
From free body diagram:
$$
\Sigma F_M = F_{CN} + F_{CT} + F_{FM} + F_{NM}
$$
Mapping from code to each of these terms:

### $F_{CN}$

In [ ]:
debug = finger_ctrlr.debug

In [ ]:
F_CN_scal = np.expand_dims(np.array(debug['F_CNs']), [1, 2])
F_CN_scal.shape # Expand shape so can be multipled

In [ ]:
F_CN_scal

These are scalar values. They need to be multiplied by the appropriate unit vector.

In [ ]:
F_CN = np.multiply(F_CN_scal, np.array(debug['N_hats']))
F_CN

### $F_{CT}$
Follows the same process as $F_{CN}$.

In [ ]:
F_CT_scal = np.expand_dims(debug['F_CTs'], [1, 2])
F_CT_scal

In [ ]:
F_CT = np.multiply(F_CT_scal, np.array(debug['T_hats']))
F_CT

In [ ]:
F_ctrl_exp = F_CT + F_CN

### $F_{FM}$

- [ ] THINK: Is it accelerations or forces that have to be equal?

Remember, friction force makes it so that the tangential forces between the two objects are the same. That implies the following equation:
$$
F_{FM} + F_{CT} = F_{FL} + F_{GT} + F_{OT}
$$
Using $F_{FL}=-F_{FM}$:
$$
F_{FM} + F_{CT} = -F_{FM} + F_{GT} + F_{OT}
$$
Rearranging terms:
$$
\begin{aligned}
2F_{FM} &= F_{GT} + F_{OT} - F_{CT} \\
F_{FM} &= \frac{F_{GT} + F_{OT} - F_{CT}}{2}
\end{aligned}
$$

First, we'll load in the summands and make sure they are vectors using unit vectors:

In [ ]:
F_GT_scal = np.expand_dims(debug['F_GTs'], [1, 2])
F_GT = np.multiply(F_GT_scal, np.array(debug['T_hats']))

F_OT_scal = np.expand_dims(debug['F_OTs'], [1, 2])
F_OT = np.multiply(F_OT_scal, np.array(debug['T_hats']))

In [ ]:
F_FM_exp = (F_GT + F_OT - F_CT)/2

### $F_{NM}$
The $F_N$ in the log is the force felt by the link, so we want the negative of that:

In [ ]:
F_NM_scal = -np.expand_dims(debug['F_Ns'], [1, 2])
F_NM_exp = np.multiply(F_NM_scal, np.array(debug['N_hats']))

In [ ]:
F_contact_exp = F_FM_exp + F_NM_exp

In [ ]:
F_exp = F_CN + F_CT + F_FM_exp + F_NM_exp

## Simulation values
### Contact forces
The simulation just gives us the contact force in the world frame. So that's easy, for computing the whole thing.

In [ ]:
F_contact_sim_raw = logger.data()[
    log_wrapper.contact_entry_start_idx+0:log_wrapper.contact_entry_start_idx+3,:]

We have to put it on the same timescale as the debug values:

In [ ]:
times = np.array(debug['times'])

In [ ]:
F_contact_sim = scipy.interpolate.interp1d(
                                logger.sample_times(),    
                                np.expand_dims(F_contact_sim_raw.T, 2), axis=0)(times)

#### $F_{FM}$ and $F_{NM}$
That being said, we still need to know the individual components. So we'll take the projection onto the normal that the simulation calculates.

In [ ]:
n_hats_sim_raw =  logger.data()[log_wrapper.contact_entry_start_idx+8:log_wrapper.contact_entry_start_idx+11,:]
n_hats_sim = scipy.interpolate.interp1d(
                        logger.sample_times(),    
                        np.expand_dims(n_hats_sim_raw.T, 2), axis=0)(times)
n_hats_sim.shape

Calculating projection matrices:

In [ ]:
n_projs_sim = np.matmul(n_hats_sim, np.transpose(n_hats_sim, [0, 2, 1]))
n_projs_sim.shape

Calculating normal force:

In [ ]:
F_NM_sim = np.matmul(n_projs_sim, F_contact_sim)
F_FM_sim = F_contact_sim - F_NM_sim

### $F_{CT}$ and $F_{CN}$
We could do the same thing as with the expected values. But for the sake of checking things, let's use the straight controller:

In [ ]:
yhat = np.array([[[0], [1], [0]]])
zhat = np.array([[[0], [0], [1]]])
yhat.shape

In [ ]:
F_CYs = np.expand_dims(debug['F_CYs'], [1, 2])*yhat
F_CZs = np.expand_dims(debug['F_CZs'], [1, 2])*zhat
F_CYs.shape

In [ ]:
F_ctrl_sim = F_CYs + F_CZs

In [ ]:
F_sim = F_contact_sim + F_ctrl_sim

## Comparison

### All forces

In [ ]:
lw=5
plt.figure(figsize=(16,24))
plt.subplot(211)
plt.plot(times, F_sim[:,1], label="Simulation", linewidth=lw, color='lightskyblue')
plt.plot(times, F_exp[:,1], label="Expectation", linewidth=lw, color='b', linestyle='--')
plt.legend()
plt.ylim(-5, 5)
plt.title(r"$\hat y$ component")
plt.ylabel("Force (N)")
plt.xlabel("Time (s)")
plt.subplot(212)
plt.plot(times, F_sim[:,2], label="Simulation", linewidth=lw, color='lightskyblue')
plt.plot(times, F_exp[:,2], label="Expectation", linewidth=lw, color='b', linestyle='--')
plt.legend()
plt.ylim(-5, 5)
plt.title(r"$\hat z$ component")
plt.ylabel("Force (N)")
plt.xlabel("Time (s)")
plt.show()

### Control forces

In [ ]:
lw=5
plt.figure(figsize=(16,24))
plt.subplot(211)
plt.plot(times, F_ctrl_sim[:,1], label="Simulation", linewidth=lw, color='lightskyblue')
plt.plot(times, F_ctrl_exp[:,1], label="Expectation", linewidth=lw, color='b', linestyle='--')
plt.ylim(-5, 5)
plt.legend()
plt.title(r"$\hat y$ component")
plt.ylabel("Force (N)")
plt.xlabel("Time (s)")
plt.subplot(212)
plt.plot(times, F_ctrl_sim[:,2], label="Simulation", linewidth=lw, color='lightskyblue')
plt.plot(times, F_ctrl_exp[:,2], label="Expectation", linewidth=lw, color='b', linestyle='--')
plt.ylim(-5, 5)
plt.legend()
plt.title(r"$\hat z$ component")
plt.ylabel("Force (N)")
plt.xlabel("Time (s)")
plt.show()

### Contact forces

In [ ]:
lw=5
plt.figure(figsize=(16,24))
plt.subplot(211)
plt.plot(times, F_contact_sim[:,1], label="Simulation", linewidth=lw, color='lightskyblue')
plt.plot(times, F_contact_exp[:,1], label="Expectation", linewidth=lw, color='b', linestyle='--')
plt.ylim(-5, 5)
plt.legend()
plt.title(r"$\hat y$ component")
plt.ylabel("Force (N)")
plt.xlabel("Time (s)")
plt.subplot(212)
plt.plot(times, F_contact_sim[:,2], label="Simulation", linewidth=lw, color='lightskyblue')
plt.plot(times, F_contact_exp[:,2], label="Expectation", linewidth=lw, color='b', linestyle='--')
plt.ylim(-5, 5)
plt.legend()
plt.title(r"$\hat z$ component")
plt.ylabel("Force (N)")
plt.xlabel("Time (s)")
plt.show()

### Normal forces

In [ ]:
lw=5
plt.figure(figsize=(16,24))
plt.subplot(211)
plt.plot(times, F_NM_sim[:,1], label="Simulation", linewidth=lw, color='lightskyblue')
plt.plot(times, F_NM_exp[:,1], label="Expectation", linewidth=lw, color='b', linestyle='--')
plt.ylim(-5, 5)
plt.legend()
plt.title(r"$\hat y$ component")
plt.ylabel("Force (N)")
plt.xlabel("Time (s)")
plt.subplot(212)
plt.plot(times, F_NM_sim[:,2], label="Simulation", linewidth=lw, color='lightskyblue')
plt.plot(times, F_NM_exp[:,2], label="Expectation", linewidth=lw, color='b', linestyle='--')
plt.ylim(-5, 5)
plt.legend()
plt.title(r"$\hat z$ component")
plt.ylabel("Force (N)")
plt.xlabel("Time (s)")
plt.show()

### Friction forces

In [ ]:
lw=5
plt.figure(figsize=(16,24))
plt.subplot(211)
plt.plot(times, F_FM_sim[:,1], label="Simulation", linewidth=lw, color='lightskyblue')
plt.plot(times, F_FM_exp[:,1], label="Expectation", linewidth=lw, color='b', linestyle='--')
plt.ylim(-5, 5)
plt.legend()
plt.title(r"$\hat y$ component")
plt.ylabel("Force (N)")
plt.xlabel("Time (s)")
plt.subplot(212)
plt.plot(times, F_FM_sim[:,2], label="Simulation", linewidth=lw, color='lightskyblue')
plt.plot(times, F_FM_exp[:,2], label="Expectation", linewidth=lw, color='b', linestyle='--')
plt.ylim(-5, 5)
plt.legend()
plt.title(r"$\hat z$ component")
plt.ylabel("Force (N)")
plt.xlabel("Time (s)")
plt.show()

## Conclusions
All forces here are correct except for friction.

What are the things that could make friction wrong?
1. My modeling relating $F_{OT}$, $F_{GT}$, and $F_{CT}$ to the friction force is wrong.
2. I'm calculating one of these forces incorrectly.

I'm suscious these are related to each other, and given that $F_{OT}$ and $F_{ON}$ are calcuted differently, even if I can get normal forces to line up, that doesn't necessarily mean that the tangential forces will line up.

Wait. Right now, the system isn't even applying stiffness or damping. So there should be no $F_ON$.

Alright, so that means these problems both need to be solved simultaneously before I can get results. To debug object forces, however, I need to have a set up to debug objects on the link. So that's what I'll look at now.

# Debug plots

In [ ]:
# Get debug dict
debug = finger_ctrlr.debug
times = np.array(debug['times']) # np.arange(len(debug['d_Ns']))*effective_tspan/len(debug['d_Ns'])
dt = effective_tspan/len(debug['d_Ns'])

## Unit vectors

In [ ]:
N_Hats = np.array(debug['N_hats'])
T_Hats = np.array(debug['T_hats'])

In [ ]:
N_Hats_sim = logger.data()[log_wrapper.contact_entry_start_idx+8:log_wrapper.contact_entry_start_idx+11,:]

In [ ]:
plt.figure(figsize=(16, 16))
lw = 5
plt.subplot(211)
plt.plot(logger.sample_times(), N_Hats_sim[0,:], label='$\hat n_{BA_W,x}$', color='pink', linewidth=lw)
plt.plot(logger.sample_times(), N_Hats_sim[1,:], label='$\hat n_{BA_W,y}$', color='lightgreen', linewidth=lw)
plt.plot(logger.sample_times(), N_Hats_sim[2,:], label='$\hat n_{BA_W,z}$', color='lightskyblue', linewidth=lw)
plt.plot(times, N_Hats[:,0,:], '--', label='$\hat N_x$', color='red', linewidth=lw)
plt.plot(times, N_Hats[:,1,:], '--', label='$\hat N_y$', color='green', linewidth=lw)
plt.plot(times, N_Hats[:,2,:], '--', label='$\hat N_z$', color='blue', linewidth=lw)
plt.xlabel("Time (seconds)")
plt.legend()
plt.subplot(212)
plt.plot(times, T_Hats[:,0,:], label='$\hat T_x$', color='red', linewidth=lw)
plt.plot(times, T_Hats[:,1,:], label='$\hat T_y$', color='green', linewidth=lw)
plt.plot(times, T_Hats[:,2,:], label='$\hat T_z$', color='blue', linewidth=lw)
plt.xlabel("Time (seconds)")
plt.legend()
plt.show()

### Projection matrices
Gives a $k\times 3 \times 3$ array that can be used with `matmul` and vectors stacked in a $k\times 3 \times 1$ shape.

In [ ]:
# Transpose with [0, 2, 1] swaps only the last two rows
N_hat_proj_mats = np.matmul(N_Hats, np.transpose(N_Hats, [0, 2, 1]))
T_hat_proj_mats = np.matmul(T_Hats, np.transpose(T_Hats, [0, 2, 1]))

In [ ]:
N_hat_proj_mats_lt = scipy.interpolate.interp1d(times, N_hat_proj_mats,
                                                  axis=0)(logger.sample_times())
T_hat_proj_mats_lt = scipy.interpolate.interp1d(times, T_hat_proj_mats,
                                                  axis=0)(logger.sample_times())

## $d_T$ and $d_N$

In [ ]:
separation_speed = logger.data()[log_wrapper.contact_entry_start_idx+3,:]
slip_speed = logger.data()[log_wrapper.contact_entry_start_idx+4,:]

In [ ]:
lw=1

plt.figure(figsize=(12, 18))
plt.subplot(311)
plt.plot(times, np.array(debug['d_Ns']), label='$d_N$', linewidth=lw)
plt.plot(times, debug['d_Ts'], label='$d_T$', linewidth=lw)
plt.axhline(constants.FINGER_RADIUS, label='Min $d_N$', linestyle='--', color='k', linewidth=lw)
plt.legend()
plt.xlabel("Time (seconds)")
plt.ylabel("Distance (m)")

plt.subplot(312)
plt.plot(logger.sample_times(), separation_speed, label="Separation speed", linewidth=lw)
plt.legend()
plt.xlabel("Time (seconds)")
plt.ylabel("Velocity (m/s)")

plt.subplot(313)
plt.plot(logger.sample_times(), slip_speed, label="slip speed")
plt.axhline(v_stiction, linestyle="--", color="k", label="$v_{stiction}$")
plt.legend()
plt.xlabel("Time (seconds)")
plt.ylabel("Speed (m/s)")
plt.ylim(0, 2*v_stiction)
plt.show()

In [ ]:
plt.figure(figsize=(12, 12))
step = 100

lw=5

plt.plot(times[:-step:step], 
         np.abs(np.diff(debug['d_Ts'][::step])/np.diff(times[::step])/100), 
         label='$\left|\\dot d_T\\right|/100$', linewidth=lw)
plt.plot(times[:-step:step], 
         np.abs(np.diff(debug['d_coms'][::step])/np.diff(times[::step])/10), 
         label='$\left|\\dot d_{comT}\\right|/10$', linewidth=lw)


plt.plot(logger.sample_times(), slip_speed*10, label="slip speed $\\times 10$", linewidth=lw)

plt.axhline(v_stiction*10, color='k', label="$v_{stiction}\\times 10$", linestyle='--', linewidth=lw)
plt.legend()
plt.ylim(0, 0.02)
plt.xlim(logger.sample_times()[0], logger.sample_times()[-1])
plt.ylabel("Speed (m/s)")

plt.show()

In [ ]:
plt.figure(figsize=(12, 12))
lw = 5
plt.plot(times, debug['d_Ts'], label='$d_T$', linewidth=lw)
plt.plot(times, debug['d_com_Ts'], label='$d_{comT}$', linewidth=lw)
plt.plot(times, debug['d_coms'], label='$d_{com}$', linewidth=lw)

plt.legend()
plt.ylabel("Distance (m)")
plt.xlim(logger.sample_times()[0], logger.sample_times()[-1])

plt.show()

## Positions
### Link state

In [ ]:
plt.figure(figsize=(12, 12))
plt.plot(times, debug['omega_xs'], label='$\omega_x$')
plt.plot(times, np.array(debug['theta_xs']), label='$\\theta_x$')
plt.legend()
plt.ylim(-2*np.pi, 2*np.pi)
plt.yticks(np.arange(-2, 2.5, 0.5)*np.pi, [
    '$-2\pi$',
    '$-3\pi/2$',
    '$-\pi$',
    '$-\pi/2$',
    '$0$',
    '$\pi/2$',
    '$\pi$',
    '$3\pi/2$',
    '$2\pi$',
])
plt.xlabel("Time (seconds)")
plt.axhline(np.pi, linestyle='--', color='k')
plt.axhline(-np.pi, linestyle='--', color='k')
plt.show()

#### Rotation matrices

In [ ]:
thetas = np.array(debug['theta_xs'])

In [ ]:
# Rotating theta about the x axis
Rs = np.zeros((thetas.size, 3, 3))
Rs[:, 1, 1] = np.cos(thetas)
Rs[:, 1, 2] = -np.sin(thetas)
Rs[:, 2, 1] = np.cos(thetas)
Rs[:, 2, 2] = np.sin(thetas)
R_invs = np.transpose(Rs, [0, 2, 1])

### Manipulator state

In [ ]:
finger_idx = int(finger_body.index())

In [ ]:
man_point = logger.data()[log_wrapper.entries_per_body*finger_idx:log_wrapper.entries_per_body*finger_idx+3]

In [ ]:
plt.figure(figsize=(12, 12))
lw=5
# plt.plot(logger.sample_times(), logger.data()[
#     log_wrapper.entries_per_body*finger_idx], label="$x$", linewidth=lw, linestyle='--')
plt.plot(logger.sample_times(), logger.data()[
    log_wrapper.entries_per_body*finger_idx+1], label="$y$", linewidth=lw)
plt.plot(logger.sample_times(), logger.data()[
    log_wrapper.entries_per_body*finger_idx+2], label="$z$", linewidth=lw)
plt.plot(logger.sample_times(), logger.data()[
    log_wrapper.entries_per_body*finger_idx+3], label="$\\theta_x$", linewidth=lw, linestyle='--')
plt.plot(logger.sample_times(), logger.data()[
    log_wrapper.entries_per_body*finger_idx+4], label="$\\theta_y$", linewidth=lw, linestyle='dashdot')
plt.plot(logger.sample_times(), logger.data()[
    log_wrapper.entries_per_body*finger_idx+5], label="$\\theta_z$", linewidth=lw, linestyle=':')
plt.legend()
plt.xlabel("Time (seconds)")
plt.ylabel("Position (m or radians)")
plt.show()

### Contact point
 - [ ] PROGRAMMING: Add arrow heads to contact point plots

In [ ]:
contact_point = logger.data()[log_wrapper.contact_entry_start_idx+5:log_wrapper.contact_entry_start_idx+8,:]

In [ ]:
contact_point.shape

In [ ]:
plt.figure(figsize=(12,12))

plt.plot(contact_point[1,:], contact_point[2,:], color='k', linestyle='--', linewidth='5')
plt.gca().set_aspect('equal', 'box')
plt.show()

In [ ]:
plt.figure(figsize=(12,12))
plt.gca().add_patch(plt.Circle((0, 0), constants.FINGER_RADIUS, color='r'))
plt.plot(contact_point[1,:]-man_point[1,:], contact_point[2,:]-man_point[2,:], color='k', linestyle='--', linewidth='5')
plt.gca().set_aspect('equal', 'box')
plt.show()

#### Contact point vs. speeds

In [ ]:
v_contact_point_no_interp = (np.diff(contact_point)/logger.sample_times()[:-1]).T

In [ ]:
v_contact_point = scipy.interpolate.interp1d(
    logger.sample_times()[:-1], v_contact_point_no_interp, axis=0, fill_value="extrapolate")(times)

In [ ]:
# Check interpolation
lw=3
plt.figure(figsize=(12,12))

plt.subplot(311)
plt.plot(times, v_contact_point[:,0],linewidth=lw)
plt.plot(logger.sample_times()[:-1], v_contact_point_no_interp[:,0],linewidth=lw, linestyle='--')
plt.xlabel("Time (s)")
plt.ylabel("Velocity (m/s)")
plt.title("$\hat x$ component")

plt.subplot(312)
plt.plot(times, v_contact_point[:,1],linewidth=lw)
plt.plot(logger.sample_times()[:-1], v_contact_point_no_interp[:,1],linewidth=lw, linestyle='--')
plt.xlabel("Time (s)")
plt.ylabel("Velocity (m/s)")
plt.title("$\hat y$ component")

plt.subplot(313)
plt.plot(times, v_contact_point[:,2],linewidth=lw)
plt.plot(logger.sample_times()[:-1], v_contact_point_no_interp[:,2],linewidth=lw, linestyle='--')
plt.xlabel("Time (s)")
plt.ylabel("Velocity (m/s)")
plt.title("$\hat z$ component")

plt.show()

They match up, so we've interpolated correctly.

In [ ]:
# Convert to matrix in manipulator basis
M_v_contact_point = np.expand_dims(v_contact_point, 2)

In [ ]:
# Change to compliance basis
C_v_contact_point = np.matmul(R_invs, M_v_contact_point)

In [ ]:
lw=3
plt.figure(figsize=(16,24))

plt.subplot(311)
plt.plot(times, C_v_contact_point[:,0],linewidth=lw, label=r"$_Cv_{contact, x}$")
plt.xlabel("Time (s)")
plt.ylabel("Velocity (m/s)")
plt.title("$\hat x$ component")

# PROGRAMMING: Is slip speed properly filled with NaNs?
plt.subplot(312)
plt.plot(times, C_v_contact_point[:,1],linewidth=lw, label=r"$_Cv_{contact, T}$")
ylims = plt.ylim()
plt.plot(logger.sample_times(), slip_speed, linewidth=lw, label="Slip speed")
plt.axhline(v_stiction, color='k', linestyle='--', label=r'$v_{stiction}$')
plt.ylim(ylims)
plt.xlabel("Time (s)")
plt.ylabel("Velocity (m/s)")
plt.title("$\hat T$ component")
plt.legend()

plt.subplot(313)
plt.plot(times, np.abs(C_v_contact_point[:,2]),linewidth=lw, label=r"$_Cv_{contact, N}$")
ylims = plt.ylim()
plt.plot(logger.sample_times(), np.abs(separation_speed), linewidth=lw, label="Separation speed")
plt.axhline(0, color='k', linestyle='--')
plt.ylim(ylims)
plt.xlabel("Time (s)")
plt.ylabel("Velocity (m/s)")
plt.title("$\hat N$ component")

plt.show()

Hm. I would expect these to link up more closely. Let's double check their defintions in drake documentation:

`slip_speed` ([link](https://drake.mit.edu/doxygen_cxx/classdrake_1_1multibody_1_1_point_pair_contact_info.html#a6d2da20c96fa99be810b28ef0705214c)):
> Returns the slip speed between body A and B at contact point C.

`separation speed` ([link](https://drake.mit.edu/doxygen_cxx/classdrake_1_1multibody_1_1_point_pair_contact_info.html#a61a3bf4aed134902cb0354fa20feedb4])):
> Returns the separation speed between body A and B along the normal direction (see PenetrationAsPointPair::nhat_BA_W) at the contact point.
>
> It is defined positive for bodies moving apart in the normal direction.

Hm. I'm intrigued by `nhat_BA_W`; maybe my $\hat N$ is not what Drake is using. Let's check its definition:

## Forces

In [ ]:
# Friction force on manipulor, assuming static friction
total_m_FF = np.array(debug['F_GTs'])+np.array(debug['F_OTs'])-np.array(debug['F_CTs'])
# Total force on the link in the N hat direction 
total_l_FN = np.array(debug['F_Ns'])+np.array(debug['F_GNs'])+np.array(debug['F_ONs'])

### $\hat N$ acceleration
Take the acceleration in the $\hat N$ direction and then multiply by the mass. This is a check that the forces I'm summing in the $\hat N$ direction are actually all the forces on the object.

In [ ]:
ll_idx = paper.get_free_edge_idx()
a_vecs_no_interp = logger.data()[log_wrapper.entries_per_body*ll_idx+12:log_wrapper.entries_per_body*ll_idx+15].T
a_vecs_no_interp = np.expand_dims(a_vecs_no_interp, 2)

In [ ]:
M_a_vecs = scipy.interpolate.interp1d(logger.sample_times(), a_vecs_no_interp, axis=0)(times)
C_a_vecs = np.matmul(R_invs, M_a_vecs)
a_N = C_a_vecs[:,2]
F_from_acc = paper.link_mass*a_N

In [ ]:
a_N_projs = np.matmul(N_hat_proj_mats, M_a_vecs)
a_N_signs = np.sign(np.matmul(np.transpose(N_Hats, [0, 2, 1]), a_N_projs))
a_N_mags = np.linalg.norm(a_N_projs, axis=1)
a_N = a_N_mags.flatten()*a_N_signs.flatten()
# F_contact_N *= -1 # The "F_N" we use is on the link, not the manipulator
F_from_acc = paper.link_mass*a_N

### Contact forces

In [ ]:
M_F_contact_yz = logger.data()[log_wrapper.contact_entry_start_idx+1:log_wrapper.contact_entry_start_idx+3,:]

In [ ]:
plt.figure(figsize=(12,12))
plt.plot(logger.sample_times(), M_F_contact_yz[0,:], linewidth=5, label='$F_y$')
plt.plot(logger.sample_times(), M_F_contact_yz[1,:], linewidth=5, label='$F_z$')
plt.legend()
plt.ylim(-10, 10)
plt.show()

In [ ]:
# Add on empty row of zeros for x
M_F_contact_x = np.zeros((1, M_F_contact_yz.shape[1]))
M_F_contact_no_interp = np.expand_dims(np.vstack((M_F_contact_x, M_F_contact_yz)).T, 2)
M_F_contact = scipy.interpolate.interp1d(logger.sample_times(), M_F_contact_no_interp, axis=0)(times)

This plot verifies the interpolation:

In [ ]:
plt.figure(figsize=(12,12))
lw=5
plt.plot(logger.sample_times(), M_F_contact_yz[0,:], color='lightskyblue', label='$F_y$ before interp', linewidth=lw)
plt.plot(logger.sample_times(), M_F_contact_yz[1,:], color='pink', label='$F_z$ before interp', linewidth=lw)
plt.plot(times, M_F_contact[:,1], color='blue', linestyle='--', label='$F_y$ after interp', linewidth=lw)
plt.plot(times, M_F_contact[:,2], color='red', linestyle='--', label='$F_z$ after interp', linewidth=lw)
plt.ylim(-5, 5)
plt.legend()
plt.ylabel("Force (N)")
plt.xlabel("Time (s)")
plt.show()

In [ ]:
# Get T and N hat projections
C_F_contact = np.matmul(R_invs, M_F_contact)
F_contact_T = C_F_contact[:,1]
F_contact_N = C_F_contact[:,2]

In [ ]:
F_contact_T_projs = np.matmul(T_hat_proj_mats, M_F_contact)
F_contact_T_signs = np.sign(np.matmul(np.transpose(T_Hats, [0, 2, 1]), F_contact_T_projs))
F_contact_T_mags = np.linalg.norm(F_contact_T_projs, axis=1)
F_contact_T = F_contact_T_mags.flatten()*F_contact_T_signs.flatten()

F_contact_N_projs = np.matmul(N_hat_proj_mats, M_F_contact)
F_contact_N_signs = np.sign(np.matmul(np.transpose(N_Hats, [0, 2, 1]), F_contact_N_projs))
F_contact_N_mags = np.linalg.norm(F_contact_N_projs, axis=1)
F_contact_N = F_contact_N_mags.flatten()*F_contact_N_signs.flatten()
F_contact_N *= -1 # The "F_N" we use is on the link, not the manipulator

In [ ]:
separation_speed_interp = scipy.interpolate.interp1d(logger.sample_times(),
                                                     logger.data()[log_wrapper.contact_entry_start_idx+3,:])(
                                                        times)

In [ ]:
plt.figure(figsize=(12,12))
lw=5
plt.plot(times, F_contact_T, linewidth=5, label='$F_T$')
plt.plot(times, F_contact_N, linewidth=5, label='$F_N$')
idxs = np.logical_and(separation_speed_interp <= 0, (F_contact_N <= 0).flatten())
plt.plot(times[idxs], F_contact_N[idxs], ' *')
plt.legend()

plt.ylim(-10, 10)
plt.show()

In [ ]:
plt.figure(figsize=(12,12))
plt.plot(times, separation_speed_interp, linewidth=5)
plt.ylim(-1, 1)
plt.show()

In [ ]:
separation_speed

### Joint forces

In [ ]:
joint_force = logger.data()[log_wrapper.joint_entry_start_idx:log_wrapper.joint_entry_start_idx+6,:]
joint_force.shape

In [ ]:
plt.figure(figsize=(16, 32))
plt.subplot(211)
plt.title("Force")
plt.plot(logger.sample_times(), joint_force[0,:], label=r'$F_x$', color='red')
plt.plot(logger.sample_times(), joint_force[1,:], label=r'$F_y$', color='green')
plt.plot(logger.sample_times(), joint_force[2,:], label=r'$F_z$', color='blue')
plt.plot(times, debug['F_centripetal'], label=r'Centripetal force')
plt.xlabel("Time (seconds)")
plt.ylim(-20, 20)
plt.legend()
plt.subplot(212)
plt.title("Torque")
plt.plot(logger.sample_times(), joint_force[3,:], label=r'$\tau_x$', color='red')
plt.plot(logger.sample_times(), joint_force[4,:], label=r'$\tau_y$', color='green')
plt.plot(logger.sample_times(), joint_force[5,:], label=r'$\tau_z$', color='blue')
# plt.axvline(logger.sample_times()[1500])
# plt.axvline(logger.sample_times()[22500])
plt.ylim(-1, 1)
plt.xlabel("Time (seconds)")
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(16, 16))
plt.title("Force")
plt.plot(logger.sample_times(), joint_force[0,:], label=r'$F_x$', color='red')
plt.plot(logger.sample_times(), joint_force[1,:], label=r'$F_y$', color='green')
plt.plot(logger.sample_times(), joint_force[2,:], label=r'$F_z$', color='blue')
plt.plot(times, debug['F_centripetal'], label=r'Centripetal force')
plt.legend()
plt.ylim(-1, 1)
plt.xlabel("Time (seconds)")
plt.legend()
plt.show()

In [ ]:
max(joint_force[1,:])

In [ ]:
np.max(np.abs(joint_force[4:6,1500:22500]))

In [ ]:
M_F_O = np.expand_dims(joint_force[0:3:].T, 2)
M_F_O.shape

In [ ]:
C_F_O = np.matmul(N_hat_proj_mats_lt, M_F_O)

In [ ]:
np.linalg.norm(M_F_O,axis=1).shape

In [ ]:
plt.figure(figsize=(16, 24))
plt.title("Force")
plt.plot(logger.sample_times(), M_F_O[:,0], label=r'$F_x$', color='red')
plt.plot(logger.sample_times(), M_F_O[:,1], label=r'$F_T$', color='green')
plt.plot(logger.sample_times(), M_F_O[:,2], label=r'$F_N$', color='blue')
plt.plot(logger.sample_times(), np.linalg.norm(M_F_O,axis=1), label='Total force')
plt.plot(times, thetas)
plt.xlabel("Time (seconds)")
plt.ylim(-20, 20)
plt.legend()
plt.show()

### Tangential

In [ ]:
plt.figure(figsize=(16, 16))
lw = 5
plt.title("Tangential forces")
F_GTs = np.array(debug['F_GTs'])
F_OTs = np.array(debug['F_OTs'])
F_CTs = np.array(debug['F_CTs'])
plt.plot(times, debug['F_GTs'], label='$F_{GT}$', linewidth=lw, linestyle='--')
plt.plot(times, debug['F_OTs'], label='$F_{OT}$', linewidth=lw, linestyle='--')
plt.plot(times, debug['F_CTs'], label='$F_{CT}$', linewidth=lw, linestyle='--')
# plt.plot(times, F_GTs + F_OTs, label='$F_{GT}+F_{OT}$', linewidth=lw, linestyle='--')
plt.plot(times, total_m_FF, label='$F_F$ inferred from static friction force balance', linewidth=lw)
plt.plot(times, -2*constants.FRICTION*np.array(debug['F_Ns']), 'k--', label="Bound on friction force", linewidth=lw)
plt.plot(times, 2*constants.FRICTION*np.array(debug['F_Ns']), 'k--', linewidth=lw)
plt.plot(times, F_contact_T, label="Measured $F_F$", linewidth=lw)
plt.legend()
plt.xlabel("Time (seconds)")
plt.ylabel("$\hat T$ force (N)")
plt.ylim(-5, 5)
plt.show()

# PROGRAMMING: Let's get the actual contact forces, and possibly also compare the tangential acceleration, on this plot.

### Normal

In [ ]:
plt.figure(figsize=(16, 16))
lw = 5
plt.title("Normal forces")
plt.plot(times, debug['F_CNs'], label='$F_{CN}$', linewidth=lw)
plt.plot(times, debug['F_GNs'], label='$F_{GN}$', linewidth=lw)
plt.plot(times, debug['F_ONs'], label='$F_{ON}$', linewidth=lw)
plt.plot(times, total_l_FN, label='Total $\hat N$ force on link', linewidth=lw)
plt.plot(times, F_from_acc, label='Total force, according tto acceleration', linewidth=lw)
plt.plot(times, F_contact_N, label='Actual F_N', linewidth=lw)
# plt.axhline(finger_ctrlr.F_Nd, color='k', linestyle='--', label='Target force')
plt.ylim(-finger_ctrlr.F_Nd*2, finger_ctrlr.F_Nd*2)
plt.legend()
plt.xlabel("Time (seconds)")
plt.ylabel("$\hat N$ force (N)")
plt.show()

In [ ]:
plt.figure(figsize=(16, 16))
lw = 5
plt.title("Contact normal forces")
plt.plot(times, debug['F_Ns'], label='$F_{N}$', linewidth=lw)
plt.plot(times, F_contact_N, label='Actual $F_N$', linewidth=lw)
plt.ylim(-5, 5)
plt.legend()
plt.xlabel("Time (seconds)")
plt.ylabel("$\hat N$ force (N)")
plt.show()

# PROGRAMMING: Why is the force not what my setpoint is?

### Comparing magnitudes

In [ ]:
cal_contact_force = np.sqrt(np.array(debug['F_Ns'])**2 + total_m_FF**2)
sim_contact_force = np.linalg.norm(M_F_contact_yz[0:2,:], axis=0)

In [ ]:
plt.figure(figsize=(12,12))
plt.plot(times, cal_contact_force, linewidth=lw)
ylims = plt.ylim()
plt.plot(logger.sample_times(), sim_contact_force, linewidth=lw)
plt.ylim(ylims)
plt.ylabel("Force (N)")
plt.xlabel("Time (seconds)")
plt.show()

## Control forces

### Link frame

In [ ]:
plt.figure(figsize=(12, 12))
plt.plot(times, debug['F_CTs'], label='$F_{CT}$')
plt.plot(times, debug['F_CNs'], label='$F_{NT}$')
plt.legend()
plt.xlabel("Time (seconds)")
plt.show()

### Manipulator frame

In [ ]:
plt.figure(figsize=(12, 12))
plt.plot(times, debug['theta_xs'], label='$\\theta_x$')
plt.plot(times, debug['omega_xs'], label='$\omega_x$')
plt.legend()
plt.show()

In [ ]:
F_Ms = np.array(debug['F_Ms'])
plt.figure(figsize=(12, 12))
plt.plot(times, F_Ms[:,0,:], label='$F_M$ $x$ component', color='red')
plt.plot(times, F_Ms[:,1,:], label='$F_M$ $y$ component', color='green')
plt.plot(times, F_Ms[:,2,:], label='$F_M$ $z$ component', color='blue')
plt.xlabel("Time (seconds)")
plt.legend()
plt.show()